Try different learning rates for GD vs MRGD

* Use cosine similarity as metric


In [ ]:
import os 
os.chdir("..")
os.chdir("..")
from src.cpwc.multires.class_multiressolver import *
import matplotlib.pyplot as plt
import torch
from src.cpwc.tools.ptychography import Ptychography as Ptychography
from src.cpwc.tools.utils import *
torch.cuda.empty_cache()
from src.utils.cuda_pyth import empty_cuda, print_cuda_tensors
from src.utils.get_image import get_image
from src.utils.noise import add_gaussian_noise, add_poisson_noise
from src.utils.metrics import get_ring_average, F2fluxconverter
from src.utils.manage_data import save_data,unwrap_2d,extract_data
# Set seeds 
torch.manual_seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"
max_scale = 9
image,image_tensor_ = get_image("samples/potential.npy", max_scale,device)
max_probe_size = 128
cycle = [0, -1, -1, -1,  1, 1, 1]
tol = [1e-10] * 9
tol_in = [1e-15] * 9
max_shift = 32
lmbda = 1e-30
#noise = 1e30
linOperator = Ptychography(max_scale = max_scale,max_probe_size = max_probe_size ,max_shift = max_shift,device=device)
multires = MultiRes(max_scale, device)
image_tensor__ = linOperator.apply(image_tensor_)
#image_tensor__ = add_poisson_noise(image_tensor__, noise, 'cuda')
loss = Loss(linOperator,image_tensor__,lmbda= lmbda)
I_in = 25*np.array([1, 15, 10, 5, 10, 30, 100])
#I_out = 10*np.array([0, 0, 0, 30, 10,10,300])
I_out = 10*np.array([0, 0, 0, 3, 1,1,117])
#I_out = 10*np.array([0, 0, 0, 0, 0,0,122])

LR_list = [0,0,0,2e-2,3e-2,5e-2,1e-1]

In [ ]:
import os 
os.chdir("..")
os.chdir("..")
from src.cpwc.multires.class_multiressolver import *
import matplotlib.pyplot as plt
import torch
from src.cpwc.tools.ptychography import Ptychography as Ptychography
from src.cpwc.tools.utils import *
torch.cuda.empty_cache()
from src.utils.cuda_pyth import empty_cuda, print_cuda_tensors
from src.utils.get_image import get_image
from src.utils.noise import add_gaussian_noise, add_poisson_noise
from src.utils.metrics import get_ring_average, F2fluxconverter
from src.utils.manage_data import save_data,unwrap_2d,extract_data
# Set seeds 
torch.manual_seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"
max_scale = 9
image,image_tensor_ = get_image("samples/potential.npy", max_scale,device)
max_probe_size = 128
cycle = [0, -1, -1, -1,  1, 1, 1]
tol = [1e-10] * 9
tol_in = [1e-15] * 9
max_shift = 32
lmbda = 1e-30
#noise = 1e30
linOperator = Ptychography(max_scale = max_scale,max_probe_size = max_probe_size ,max_shift = max_shift,device=device)
multires = MultiRes(max_scale, device)
image_tensor__ = linOperator.apply(image_tensor_)
#image_tensor__ = add_poisson_noise(image_tensor__, noise, 'cuda')
loss = Loss(linOperator,image_tensor__,lmbda= lmbda)
I_in = 25*np.array([1, 15, 10, 5, 10, 30, 100])
#I_out = 10*np.array([0, 0, 0, 30, 10,10,300])
I_out = 10*np.array([0, 0, 0, 3, 1,1,117])
#I_out = 10*np.array([0, 0, 0, 0, 0,0,122])

LR_list = [0,0,0,2e-2,3e-2,5e-2,1e-1]

import torch
import numpy as np
from scipy.sparse.linalg import LinearOperator, eigs

# Define the function once more for clarity
def matvec(x, c, y):
    Ax = linOperator.apply_linop(c)            # (1, B, N, N)
    Ax_pwr = linOperator.apply(c)              # (1, B, N, N)
    abs_Ax = torch.sqrt(Ax_pwr)
    sqrt_y = torch.sqrt(y)
    
    term1_factor = (sqrt_y - abs_Ax) / (abs_Ax + 1e-10)
    term1 = term1_factor * torch.conj(Ax)
    term2 = (Ax / (abs_Ax + 1e-10)) * (torch.conj(Ax) / (abs_Ax + 1e-10))
    hterm = term1 + term2

    x = linOperator.apply_linop(x)
    x = x * hterm
    x = linOperator.apply_linopT(x)
    return x

# Build the LinearOperator
def build_operator(c, y):
    shape = c.shape
    size = int(np.prod(shape))

    def mv(x_np):
        x = torch.from_numpy(x_np).to(c.device).type(torch.cfloat).reshape_as(c)
        out = matvec(x, c, y)
        return out.detach().cpu().numpy().reshape(-1)

    return LinearOperator(
        dtype=np.complex64,
        shape=(size, size),
        matvec=mv
    )

c = torch.randn_like(image_tensor__).to(device) 
linop = build_operator(c=c, y=image_tensor__)
vals, vecs = eigs(linop, k=1, which='LM')

print("Largest eigenvalue:", vals[0])




In [ ]:
model = MultiResSolver(multires, 
                        loss, 
                        LR = LR_list,
                        I_in = I_in,
                        I_out = I_out,
                        tol = tol,
                        tol_in = tol_in,
                        cycle = cycle,
                        l1_type = "l1_row",
                        gt = image_tensor_)
model.solve_multigrid()
model.print_time()

In [ ]:
loss  = extract_data(model.measures["loss"])
plt.semilogy(loss)
plt.grid()
plt.show()

In [ ]:
mean_img = np.mean(image)
phase = torch.angle(model.c_k[0,0,:,:].to('cpu'))
phase = phase.numpy()
phase = unwrap_2d(phase)
plt.imshow(phase, cmap='gray')


In [ ]:
if max_shift == 32:
    overlap = 75
elif max_shift == 64:
    overlap = 50
else:
    overlap = 25
from src.utils.manage_data import save_data,unwrap_2d,extract_data

path = save_data(
          path = "experiments/results/exp4",
          model = model,
          image_path= "samples/Shepp_logan.png",
          metrics = ['csim','loss','frc',"image"],
          device = "cuda",
          max_scale = 9,
          overlap = overlap,
          spline_type="cpwc",
          lambda_ = lmbda,
          lr = 2e-2,
          noise_type="poisson",
          noise = "None",
          loop = "mrgdp")



In [ ]:
mrgd001 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgd_lambda1e-30_noise_typepoisson_noiseNone_lr0.001_csim.npy")
mrgdp01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgdp_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_csim.npy")
mrgd01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgd_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_csim.npy")
gd001 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.001_csim.npy")
gd01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_csim.npy")
gd1 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.1_csim.npy")

In [ ]:
plt.figure(figsize=(15,7))
plt.title("1-Cosine Similarity")
plt.semilogy(1-mrgd001,label="MRGD w/ LR= 0.001")
plt.semilogy(1-mrgd01,label="MRGD w/ LR= 0.01")
plt.semilogy(1-mrgdp01,label="MRGD w/ LR= 0.01 x 2^(scale - min_scale)")
plt.semilogy(1-gd001,label="GD w/ LR= 0.001")
plt.semilogy(1-gd01,label="GD w/ LR= 0.01")
plt.semilogy(1-gd1,label="GD w/ LR= 0.1")
plt.xlabel("Iterations")
plt.ylabel("Cosine Similarity")
plt.grid()
plt.legend()
plt.show()


In [ ]:
mrgd001 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgd_lambda1e-30_noise_typepoisson_noiseNone_lr0.001_loss.npy")
mrgd01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgd_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_loss.npy")
mrgdp01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgdp_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_loss.npy")
gd001 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.001_loss.npy")
gd01 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.01_loss.npy")
gd1 = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.1_loss.npy")

In [ ]:
plt.figure(figsize=(10,5))
plt.title("MSE")
plt.semilogy(mrgd001,label="MRGD w/ LR= 0.001")
plt.semilogy(mrgd01,label="MRGD w/ LR= 0.01")
plt.semilogy(mrgdp01,label="MRGD w/ LR= 0.01 x 2^(scale - min_scale)")
plt.semilogy(gd001,label="GD w/ LR= 0.001")
plt.semilogy(gd01,label="GD w/ LR= 0.01")
plt.semilogy(gd1,label="GD w/ LR= 0.1")
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.grid()
plt.legend()
plt.show()


In [ ]:
gd = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.1_loss.npy")
mrgd = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp1/cpwc_overlap50_mrgdp_lambda1e-30_noise_typepoisson_noiseNone_lr0.02_loss.npy")

In [ ]:
plt.figure()
plt.title("MSE")
plt.semilogy(gd,label="GD w/ LR= 0.1")
plt.semilogy(mrgd,label="MRGD w/ LR= 0.02")
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.grid()
plt.legend()
plt.show()

In [ ]:
img = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp4/cpwc_overlap75_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.02_image.npy")
plt.figure(dpi = 600)
plt.imshow(img, cmap='gray')
plt.axis('off')


In [ ]:
lossmrgd = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp4/cpwc_overlap75_mrgdp_lambda1e-30_noise_typepoisson_noiseNone_lr0.02_loss.npy")
lossgd = np.load("/home/efe/Desktop/Multiresolution-Framework-for-Fourier-Ptychography/experiments/results/exp4/cpwc_overlap75_gd_lambda1e-30_noise_typepoisson_noiseNone_lr0.02_loss.npy")


In [ ]:
plt.figure(figsize=(10,5))
plt.title("MSE")
plt.semilogy(lossmrgd,label="MRGD")
plt.semilogy(lossgd,label="GD")
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.grid()
plt.legend()
plt.show()
